<a href="https://colab.research.google.com/github/RoyceLam/Feed_reader_Project/blob/master/Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn import preprocessing
import numpy as np
from simpletransformers.classification import ClassificationModel

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [42]:
tf.__version__

'2.3.0'

In [14]:
from google.colab import files
uploaded = files.upload()

Saving HK_classified_HK.csv to HK_classified_HK.csv


In [73]:
df=pd.read_csv('HK_classified_HK.csv')

In [74]:
df.head()

,Account Name,Account Number,Value Date,Posting Date,Account Currency,Transaction Amount,USD Equivalent,Tranx Type,Transaction Narrative,Bank Reference,Customer Reference,Bank
0,ACAL,580-067254-001,14/04/2020,14/04/2020,HKD,"29,482.20",NaN,receipt,POLAR HK LTD AUTOPAY OUT L02AUTO CREDIT HSBCHK...,NONREF,AUTOPAY,HSBC
1,ACAL,580-067254-001,14/04/2020,14/04/2020,HKD,"14,906.00",NaN,receipt,CHEQUE DEPOSIT REF:5842 0333 REF:5842 0333 CQM...,NaN,CHK,HSBC
2,ACAL,580-067254-001,14/04/2020,14/04/2020,HKD,"114,004.80",NaN,receipt,CHEQUE DEPOSIT REF:5842 0334 REF:5842 0334 CQM...,NaN,CHK,HSBC
3,ACAL,580-067254-001,14/04/2020,14/04/2020,HKD,"171,873.40",NaN,receipt,"DONGGUAN HUBBELL ELECTRICAL /OCMT/HKD171873,40...",HK114040QL99WR7K,24001TT202002114,HSBC
4,ACAL,580-067254-001,14/04/2020,14/04/2020,HKD,"312,775.80",NaN,receipt,CR ADV,MEDI,STS PYT-CHARTER,HSBC


In [75]:
df['totalwords'] = df['Transaction Narrative'].str.split().str.len()

In [76]:
df['totalwords'].max()

31

In [30]:
x=[]

In [31]:
x=df['Tranx Type'].unique()

In [32]:
x

array(['receipt', 'IC', 'expense', 'sweep', 'receipt_tw', 'receipt_kr',
       'IC-trade non apac', 'Collection from marubun', 'ic', 'vendor',
       'factoring', 'IC-non apac non trade', 'fx', 'FX', 'IC-non apac',
       'factoring receipt', 'Receipt_tw', 'deposit', 'salary', 'BMGint',
       'factoring expense', 'n-IC', 'non ic'], dtype=object)

In [77]:
le = preprocessing.LabelEncoder() 
le.fit(np.unique(df['Tranx Type'].tolist()))

LabelEncoder()

In [78]:
train_df = pd.DataFrame(df[['Tranx Type','Transaction Narrative']])

In [79]:
train_df

,Tranx Type,Transaction Narrative
0,receipt,POLAR HK LTD AUTOPAY OUT L02AUTO CREDIT HSBCHK...
1,receipt,CHEQUE DEPOSIT REF:5842 0333 REF:5842 0333 CQM...
2,receipt,CHEQUE DEPOSIT REF:5842 0334 REF:5842 0334 CQM...
3,receipt,"DONGGUAN HUBBELL ELECTRICAL /OCMT/HKD171873,40..."
4,receipt,CR ADV
...,...,...
11149,BMGint,Debit Interest
11150,BMGint,Debit Interest
11151,IC-non apac,FT201468W7ZP NL73BKMG0261243519 ARROW ELECTRON...
11152,factoring receipt,FT20149F2LTW 24662 ARROW ELECTRONICS CHINA LTD...


In [80]:
le = preprocessing.LabelEncoder() 
le.fit(np.unique(train_df['Tranx Type'].tolist()))

LabelEncoder()

In [81]:
train_df[2] = train_df['Tranx Type'].apply(lambda x:le.transform([x])[0])

In [82]:
train_df

,Tranx Type,Transaction Narrative,2
0,receipt,POLAR HK LTD AUTOPAY OUT L02AUTO CREDIT HSBCHK...,17
1,receipt,CHEQUE DEPOSIT REF:5842 0333 REF:5842 0333 CQM...,17
2,receipt,CHEQUE DEPOSIT REF:5842 0334 REF:5842 0334 CQM...,17
3,receipt,"DONGGUAN HUBBELL ELECTRICAL /OCMT/HKD171873,40...",17
4,receipt,CR ADV,17
...,...,...,...
11149,BMGint,Debit Interest,0
11150,BMGint,Debit Interest,0
11151,IC-non apac,FT201468W7ZP NL73BKMG0261243519 ARROW ELECTRON...,4
11152,factoring receipt,FT20149F2LTW 24662 ARROW ELECTRONICS CHINA LTD...,12


In [83]:
train_df=train_df.drop(columns='Tranx Type')

In [84]:
train_df

,Transaction Narrative,2
0,POLAR HK LTD AUTOPAY OUT L02AUTO CREDIT HSBCHK...,17
1,CHEQUE DEPOSIT REF:5842 0333 REF:5842 0333 CQM...,17
2,CHEQUE DEPOSIT REF:5842 0334 REF:5842 0334 CQM...,17
3,"DONGGUAN HUBBELL ELECTRICAL /OCMT/HKD171873,40...",17
4,CR ADV,17
...,...,...
11149,Debit Interest,0
11150,Debit Interest,0
11151,FT201468W7ZP NL73BKMG0261243519 ARROW ELECTRON...,4
11152,FT20149F2LTW 24662 ARROW ELECTRONICS CHINA LTD...,12


In [85]:
num_labels =len(np.unique(train_df[2].tolist()))
num_labels

23

In [93]:
model = ClassificationModel(
    'distilbert', 
    'distilbert-base-uncased-distilled-squad',
    num_labels=num_labels,
    use_cuda=False,
    args={"reprocess_input_data": True,   # 对输入数据进行预处理
          "overwrite_output_dir": True}   # 可覆盖输出文件夹
    )

Some weights of the model checkpoint at distilbert-base-uncased-distilled-squad were not used when initializing DistilBertForSequenceClassification: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-distilled-squad and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-

In [ ]:
model.train_model(train_df,args = {'fp16':False})

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:351: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
